In [1]:
import cv2

In [2]:
cap = cv2.VideoCapture('09-18-2025_0800PM.mp4')
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Should show total frames
ret, frame = cap.read()
print(ret, frame.shape if ret else "Failed")

53970.0
True (1536, 3840, 3)


In [3]:
cap = cv2.VideoCapture('09-18-2025_0800PM_clean.mp4')
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Should show total frames
ret, frame = cap.read()
print(ret, frame.shape if ret else "Failed")

44665.0
True (1536, 3840, 3)


In [4]:
cap = cv2.VideoCapture('09-18-2025_0830PM.mp4')
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Should show total frames
ret, frame = cap.read()
print(ret, frame.shape if ret else "Failed")

0.0
False Failed


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x101ece160] moov atom not found
OpenCV: Couldn't read video stream from file "09-18-2025_0830PM.mp4"
